In [ ]:
import numpy as np
import pytorch_lightning as pl
import torch
import wandb
import yaml
import pickle
from pytorch_lightning.callbacks import EarlyStopping
from sklearn.metrics import (mean_absolute_error,
                             mean_absolute_percentage_error,
                             mean_squared_error, r2_score)

from src.models import get_model
from src.model_training import TrainingLoop

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
PROJECT_NAME = "machine-learning-course-test"

In [ ]:
def init_params():
    with open("../experiments/configs/config_defaults.yaml", "r") as f:
        try:
            yaml_file = yaml.safe_load(f)
            defaults = dict(yaml_file)
            return defaults
        except yaml.YAMLError as exc:
            print(exc)


def load_data():
    train = torch.load("../data/train.pt")
    val = torch.load("../data/val.pt")
    test = torch.load("../data/test.pt")

    return train, val, test

In [ ]:
def tuning():

    wandb.init(config=init_params(), project=PROJECT_NAME, allow_val_change=True)
    params = wandb.config

    train, val, test = load_data()

    # Non-tunable parameters
    input_size = next(iter(train))[0].shape[0]

    model = get_model(params['model'])
    model = model(input_size=input_size, device=DEVICE, **params)
    print(model)

    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=6,
        mode='min'
    )

    loop = TrainingLoop(
        model=model,
        datasets=(train, val, test),
        learning_rate=params['lr'],
        batch_size=params['batch_size'],
        optimizer=params['optimizer'],
        accelerator=DEVICE,
        train_shuffle=params['train_dl_shuffle'],
        track_wandb=True,
        num_dl_workers=0
    )

    trainer = pl.Trainer(
        accelerator='cpu',
        devices=1,
        min_epochs=1,
        max_epochs=params['epochs'],
        callbacks=[early_stopping],
        log_every_n_steps=1
    )

    trainer.fit(loop)
    trainer.test(ckpt_path='best')

    predictions, labels = loop.predictions, loop.values
    if DEVICE == 'cuda':
        predictions = [tensor.cpu() for tensor in predictions]
        labels = [tensor.cpu() for tensor in labels]
    
    scaler = pickle.load(open('../data/scaler.pkl', 'rb'))
    labels = scaler.inverse_transform((np.concatenate(labels, axis=0)))
    predictions = scaler.inverse_transform((np.concatenate(predictions, axis=0)))

    mae = mean_absolute_error(labels, predictions)
    rmse = mean_squared_error(labels, predictions)**0.5
    r2 = r2_score(labels, predictions)
    mape = mean_absolute_percentage_error(labels, predictions) * 100
    
    wandb.log({
        'pred_mae': mae,
        'pred_rmse': rmse,
        'pred_r2': r2,
        'pred_mape': mape
    })


In [ ]:
with open("../experiments/configs/random_tuning_sweep.yaml", "r", encoding='utf-8') as f:
    file = yaml.safe_load(f)
    sweep_config = dict(file)

sweep_id = wandb.sweep(sweep_config, project=PROJECT_NAME)

wandb.agent(sweep_id, function=tuning, count=5)

wandb.finish()